# Project 4
## Students:
 > [Tanner Mengel]
 > [Ian Cox]
 
 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import re

2023-05-01 08:57:41.755335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 08:57:42.647138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-01 08:57:42.647188: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-01 08:57:45.476406: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

In [2]:
print(tf.__version__)# you may want to upgrade to 2.10.0 

2.11.0


In [3]:
lines = np.loadtxt('beatles.txt',dtype='str',delimiter='\n')
lyrics = []
for l in lines:
    for w in l.split():
        lyrics.append(w)
lyrics = np.array(lyrics)

### Please Use Markdown
> for markdown, see here: https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

## Task 1

In [77]:
class TransformerModel():
    def __init__(self, vocab_size, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=80, rate=0.1):
        #initailize variables
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.ff_dim = ff_dim
        self.maxlen = maxlen
        self.rate = rate

    def TransformerBlock(self, inputs):
        #create the transformer block as discribed in the writeup, use the Keras functional API (https://keras.io/guides/functional_api/)
        #MultiHeadAttention layer, specifiy 'use_causal_mask=True' (https://keras.io/api/layers/attention_layers/multi_head_attention/)
        MHA_Layer = layers.MultiHeadAttention(num_heads=self.num_heads,  key_dim=self.embed_dim,  dropout=self.rate)(inputs, inputs, use_causal_mask=True)
        
        #Dropout layer
        dropout_1 = layers.Dropout(rate=self.rate)(MHA_Layer)
        #LayerNormalization layer, specifiy 'epsilon=1e-6' (https://keras.io/api/layers/normalization_layers/layer_normalization/)
        normlayer_1 = layers.LayerNormalization(epsilon=1e-6)(inputs + dropout_1)
        #Use the rate variable for the dropout layers and remember to use two dense layers
        # Feed-Forward Layer
        dense_1 = layers.Dense(units=self.ff_dim, activation='relu')(normlayer_1)
        # Second Dense layer
        dense_2 = layers.Dense(units=self.ff_dim)(dense_1)
        # Second Dropout layer
        dropout_2 = layers.Dropout(rate=self.rate)(dense_2)
        # Output Layer norm
        transformer_output = layers.LayerNormalization(epsilon=1e-6)(normlayer_1 + dropout_2)
        #See assignment and its figures for more details.
        return transformer_output

    
    def EmbeddingLayer(self, inputs):
        #create the embedding layer
        #create (1) an embedding for the tokens and (2) an embedding for the positions
        #you can use https://keras.io/api/layers/core_layers/embedding/ Embedding class
        #you can use tf.range to enocde positions
        #add (1) and (2) and return the layer
        embedding_tokens = layers.Embedding(input_dim=self.vocab_size, output_dim=self.embed_dim)(inputs)
        embedding_positions = layers.Embedding(input_dim=self.maxlen, output_dim=self.embed_dim)(tf.range(start=0, limit=self.maxlen, delta=1))
        embeddings = embedding_tokens + embedding_positions
        return embeddings

    
    def create_model(self):
        #combine the EmbeddingLayer and num_blocks TransformerBlocks to create the model, use the Keras functional API (https://keras.io/guides/functional_api/)
        #use the SparseCategoricalCrossentropy loss function (https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class)
         # Model Input
        input = layers.Input(shape=(self.maxlen,))
        # Embedding Layer
        embeddings = self.EmbeddingLayer(input)
        # Transformer Blocks
        for i in range(self.num_blocks):
            embeddings = self.TransformerBlock(embeddings)
        # Output Layer
        model_output = layers.Dense(units=self.vocab_size, activation='softmax')(embeddings)
        # Create Model
        model = tf.keras.Model(inputs=input, outputs=model_output)
        # Compile Model
        model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
        return model

## Task 2

In [71]:
class DataSet():
    def __init__(self, filename, len):
        #load the text from the file
        with open(filename, 'r') as f:
            self.text = f.read()
        self.len = len
        self.filename = filename

    def prep_text(self):
        #remove all punctuation, set to lowercase, remove duplicate spaces and other whitespace (keep newlines)
        self.text = re.sub(r'[^\w\s]', '', self.text).lower()
        self.text = re.sub(r'\s+', ' ', self.text)
        self.text = re.sub(r'[  ]', ' ', self.text)
        
        
    def tokenize_text(self):
        #seperate into words, create a vocab and convert the text to a list of numbers using the vocab such that each unique word is represented by its own number number
        self.words = self.text.split(' ')
        self.vocab = np.unique(self.words)
        self.word2idx = {word: i for i, word in enumerate(self.vocab)}
        self.idx2word = {i: word for i, word in enumerate(self.vocab)}
        self.tokens = np.array([self.word2idx[word] for word in self.words])
        

    def create_dataset(self):
        #split the tokenized data into sequences of length len, return the sequences and vocab
        self.prep_text()
        self.tokenize_text()
        
        x, y = [], []
        for i in range(0, len(self.tokens) - self.len, self.len):
            seq_x = self.tokens[i:i+self.len]
            seq_y = self.tokens[i+1:i+self.len+1]
            x.append(seq_x)
            y.append(seq_y)
        self.x = np.array(x)
        self.y = np.array(y)
        
        return self.x, self.y, self.vocab

## Task 3

In [91]:
class GenerateText():
    def __init__(self, model, vocab):
        self.model = model
        self.vocab = {word: i for i, word in enumerate(vocab)}
        self.idx_to_word = {i: word for i, word in enumerate(self.vocab)}

    def generate_text(self, start_string, num_generate=100):
        input_eval = [self.vocab[word] for word in start_string.split(' ') if word in self.vocab]
        input_eval = tf.expand_dims(input_eval, 0)
        text_generated = []
        
        # Here, batch size == 1
        self.model.reset_states()
        for i in range(num_generate):
            predictions = self.model(input_eval)
            # Remove the batch dimension
            predictions = tf.squeeze(predictions, 0)
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(self.idx_to_word[predicted_id])

        return ' '.join(text_generated)
    
    def generate_random_text(self, num_generate=100):
        start_string = np.random.choice(list(self.vocab.keys()))
        return self.generate_text(start_string, num_generate)

## Task 4: Model Traning and Testing

In [95]:
#Train the model while periodically generating text to show progress
def train_model(model, vocab, x, y, epochs=50):
    for i in range(epochs):
        model.fit(x, y, epochs=1, batch_size=64, verbose=1)
        print("Epoch:", i + 1)
        
        # Generate text after each epoch
        generate_text = GenerateText(model, vocab)
        print(generate_text.generate_text("love", num_generate=50))
    
    return model

In [96]:
ds = DataSet('beatles.txt', len=80)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=2, maxlen=80, num_blocks=2, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=1)

8/8 [==============================] - 14s 1s/step - loss: 7.0684 - accuracy: 0.0410
Epoch: 1
rival thank picking insecure became penetrate hurry grabbed voices son going ridin maid shoulders den differnt ticking find choking trip shouts ruins gesehen much cause wanted misses tearing crying sweater loved space jack clouds stirring fill rockin hate if straight writing broke turned 8 words roses heat strawberry use thru


In [97]:
ds = DataSet('beatles.txt', len=80)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=2, maxlen=80, num_blocks=2, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=50)

8/8 [==============================] - 15s 1s/step - loss: 7.0940 - accuracy: 0.0436
Epoch: 1
movement pleas kite lords motor shaves makes eh date trail dead climb u andern admit special nancy balalaika hugs command high thin turn textpert fade missing tucson dies football heads arise wonders folks rehearsal what allein hey man rollin done froh do bath public follow knife pool bang lend bye
8/8 [==============================] - 10s 1s/step - loss: 5.9443 - accuracy: 0.0556
Epoch: 2
roses rice kite drehtest marmalade one perverted teachers boac jar flowers shaves standing across pillow end bringing command as carathon men toys spaniel quando tiger helping other papers beautiful beginning oughta moves lead worm aren moment rything mon glaubst possessing tie pennies fill reprise nearly ice fifty smiling at dies
8/8 [==============================] - 10s 1s/step - loss: 5.8559 - accuracy: 0.0564
Epoch: 3
darning bag thru teacher cares short talk bill ow tantamucho could messrs tantamucho 

In [98]:
ds = DataSet('beatles.txt', len=80)
x, y, vocab = ds.create_dataset()
model = TransformerModel(vocab_size=len(vocab), embed_dim=256, num_heads=2, maxlen=80, num_blocks=2, ff_dim=256, rate=0.1).create_model()
model = train_model(model, vocab, x, y, epochs=100)

8/8 [==============================] - 13s 1s/step - loss: 7.0574 - accuracy: 0.0420
Epoch: 1
poop born two flowers below only blackbird shoulders jar waves easy bob silly trolly inner smiling round kick foot appear win chasing possessions indicate entschuldigst choose engaged anything runs minute bags illusion world daddy across lived monkey self  aware t denkt tchaikovsky heading cocker celebrate treatin flows behave sea
8/8 [==============================] - 9s 1s/step - loss: 5.9497 - accuracy: 0.0547
Epoch: 2
dreadful mmm last strawberry sty dime hello hideaway our someone biggest shoeshine treat hears even born step child blind had sat anyway beethoven that alone kindness falling number if garden own obscene pablo cream turns sees feeling universe thank aaaaaaaahhhh kave been broken go jukebox every go pam mad moments
8/8 [==============================] - 9s 1s/step - loss: 5.8519 - accuracy: 0.0604
Epoch: 3
twice penetrate against molly blink hung deine textpert brown down week


# Report

## Introduction

## Results

## Conclusion

## How to Run Code

Please include any special libraries and list your tf version here.